<a href="https://colab.research.google.com/github/Jyothiraditya135/Some-Codes/blob/main/SetFit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q setfit

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 38.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 51.3 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
!wget https://github.com/ncbi-nlp/BLUE_Benchmark/releases/download/0.1/data_v0.1.zip
!unzip data_v0.1.zip

In [ ]:
# Read train/test files
test_df = pd.read_csv('/content/data/hoc/test.tsv', sep='\t')
train_df = pd.read_csv('/content/data/hoc/train.tsv', sep='\t')

In [ ]:
test_df.head(32)

,index,sentence,labels
0,11872299_s0,PURPOSE We recently reported that overexpressi...,NaN
1,11872299_s1,Because cyclin D1 is a downstream sensor of EG...,NaN
2,11872299_s2,We further investigated the influence of radia...,NaN
3,11872299_s3,METHODS AND MATERIALS Cyclin D1 expression was...,NaN
4,11872299_s4,These tumors greatly differed in their radiore...,NaN
5,11872299_s5,The expression of cyclin D1 and p27 proteins w...,NaN
6,11872299_s6,Cell proliferative activity in tumors was dete...,NaN
7,11872299_s7,The effect of irradiation on the expression of...,NaN
8,11872299_s8,RESULTS Cyclin D1 expression varied among tumo...,sustaining proliferative signaling
9,11872299_s9,The level of cyclin D1 expression paralleled t...,sustaining proliferative signaling


In [ ]:
LABELS = ['activating invasion and metastasis', 'avoiding immune destruction',
          'cellular energetics', 'enabling replicative immortality', 'evading growth suppressors',
          'genomic instability and mutation', 'inducing angiogenesis', 'resisting cell death',
          'sustaining proliferative signaling', 'tumor promoting inflammation']

In [ ]:
# Convert labels to hotvec multilabel format (similar to scikit-learn)
def hotvec_multilabel(true_df):
    data = {}

    for i in range(len(true_df)):
        true_row = true_df.iloc[i]

        key = true_row['index']

        data[key] = set()

        if not pd.isna(true_row['labels']):
            for l in true_row['labels'].split(','):
                data[key].add(LABELS.index(l))

    y_hotvec = []
    for k, (true) in data.items():
        t = [0] * len(LABELS)
        for i in true:
            t[i] = 1

        y_hotvec.append(t)

    y_hotvec = np.array(y_hotvec)

    return y_hotvec

In [ ]:
from datasets import Dataset
import evaluate
from setfit import SetFitModel, SetFitTrainer

In [ ]:
model = SetFitModel.from_pretrained(
    "sentence-transformers/paraphrase-mpnet-base-v2",
    multi_target_strategy="multi-output",     # one-vs-rest; multi-output; classifier-chain
)

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


In [ ]:
multilabel_f1_metric = evaluate.load("f1", "multilabel")
multilabel_accuracy_metric = evaluate.load("accuracy", "multilabel")

In [ ]:
# f1/accuracy sentence level
def compute_metrics(y_pred, y_test):
    return {
        "f1": multilabel_f1_metric.compute(predictions=y_pred, references=y_test, average="micro")["f1"],
        "accuracy": multilabel_accuracy_metric.compute(predictions=y_pred, references=y_test)["accuracy"],
    }

In [ ]:
eval_dataset = Dataset.from_dict({"text": test_df['sentence'], "label": hotvec_multilabel(test_df)})
train_dataset = Dataset.from_dict({"text": train_df['sentence'], "label": hotvec_multilabel(train_df)})

In [ ]:
print(train_dataset)

Dataset({
    features: ['text', 'label'],
    num_rows: 10527
})


In [ ]:
trainer = SetFitTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    metric=compute_metrics,
    num_iterations=5,
)

In [ ]:
trainer.train()
metrics = trainer.evaluate()
print(metrics)

In [ ]:
def divide(x, y):
    return np.true_divide(x, y, out=np.zeros_like(x, dtype=np.float64), where=y != 0)

In [ ]:
def get_p_r_f_arrary(test_predict_label, test_true_label):
    num, cat = test_predict_label.shape
    acc_list = []
    prc_list = []
    rec_list = []
    f_score_list = []
    for i in range(num):
        label_pred_set = set()
        label_gold_set = set()

        for j in range(cat):
            if test_predict_label[i, j] == 1:
                label_pred_set.add(j)
            if test_true_label[i, j] == 1:
                label_gold_set.add(j)

        uni_set = label_gold_set.union(label_pred_set)
        intersec_set = label_gold_set.intersection(label_pred_set)

        tt = len(intersec_set)
        if len(label_pred_set) == 0:
            prc = 0
        else:
            prc = tt / len(label_pred_set)

        acc = tt / len(uni_set)

        rec = tt / len(label_gold_set)

        if prc == 0 and rec == 0:
            f_score = 0
        else:
            f_score = 2 * prc * rec / (prc + rec)

        acc_list.append(acc)
        prc_list.append(prc)
        rec_list.append(rec)
        f_score_list.append(f_score)

    mean_prc = np.mean(prc_list)
    mean_rec = np.mean(rec_list)
    f_score = divide(2 * mean_prc * mean_rec, (mean_prc + mean_rec))
    return mean_prc, mean_rec, f_score

In [ ]:
def eval_hoc(true_df, pred_df):
    data = {}

    assert len(true_df) == len(pred_df), \
        f'Gold line no {len(true_df)} vs Prediction line no {len(pred_df)}'

    for i in range(len(true_df)):
        true_row = true_df.iloc[i]
        pred_row = pred_df.iloc[i]
        assert true_row['index'] == pred_row['index'], \
            'Index does not match @{}: {} vs {}'.format(i, true_row['index'], pred_row['index'])

        key = true_row['index'][:true_row['index'].find('_')]
        if key not in data:
            data[key] = (set(), set())

        if not pd.isna(true_row['labels']):
            for l in true_row['labels'].split(','):
                data[key][0].add(LABELS.index(l))

        if not pd.isna(pred_row['labels']):
            for l in pred_row['labels'].split(','):
                data[key][1].add(LABELS.index(l))

    assert len(data) == 315, 'There are 315 documents in the test set: %d' % len(data)

    y_test = []
    y_pred = []
    for k, (true, pred) in data.items():
        t = [0] * len(LABELS)
        for i in true:
            t[i] = 1

        p = [0] * len(LABELS)
        for i in pred:
            p[i] = 1

        y_test.append(t)
        y_pred.append(p)

    y_test = np.array(y_test)
    y_pred = np.array(y_pred)

    r, p, f1 = get_p_r_f_arrary(y_pred, y_test)
    print('Precision: {:.1f}'.format(p*100))
    print('Recall   : {:.1f}'.format(r*100))
    print('F1       : {:.1f}'.format(f1*100))

In [ ]:
test_predict_label = trainer.model.predict(test_df['sentence'])

In [ ]:
# Convert hotvec multilabel to actual labels
num, cat = test_predict_label.shape
sentence_list = []
for i in range(num):
    sentence_set = set()
    for j in range(cat):
        if test_predict_label[i, j] == 1:
            sentence_set.add(LABELS[j])
    sentence_list.append(','.join(sentence_set))

# Reformat for HoC evaluation
pred_df = test_df
pred_df = pred_df.assign(labels = sentence_list)
pred_df['labels'] = pred_df['labels'].replace({'':np.nan})
test_df['labels'] = test_df['labels'].replace({'':np.nan})

In [ ]:
eval_hoc(test_df, pred_df)